In [ ]:
import pandas as pd
import numpy as np

from sklearn.neighbors import NearestNeighbors

from matplotlib import pyplot as plt
import seaborn as sns


In [ ]:
# 1) FRESH: annual spending (m.u.) on fresh products (Continuous);
# 2) MILK: annual spending (m.u.) on milk products (Continuous);
# 3) GROCERY: annual spending (m.u.) on grocery products (Continuous);
# 4) FROZEN: annual spending (m.u.) on frozen products (Continuous)
# 5) DETERGENTS_PAPER: annual spending (m.u.) on detergents and paper products (Continuous)
# 6) DELICATESSEN: annual spending (m.u.) on and delicatessen products (Continuous);
# 7) CHANNEL: customers' Channel - Horeca (Hotel/Restaurant/CafÃ©) or Retail channel (Nominal)
# 8) REGION: customers' Region - Lisnon, Oporto or Other (Nominal)
# Descriptive Statistics:

# (Minimum, Maximum, Mean, Std. Deviation)
# FRESH ( 3, 112151, 12000.30, 12647.329)
# MILK (55, 73498, 5796.27, 7380.377)
# GROCERY (3, 92780, 7951.28, 9503.163)
# FROZEN (25, 60869, 3071.93, 4854.673)
# DETERGENTS_PAPER (3, 40827, 2881.49, 4767.854)
# DELICATESSEN (3, 47943, 1524.87, 2820.106)

# REGION Frequency
# Lisbon 77
# Oporto 47
# Other Region 316
# Total 440

# CHANNEL Frequency
# Horeca 298
# Retail 142
# Total 440



# 1 Loading the Data, Preprocessing, Initial Data Analysis

In [ ]:
df = pd.read_csv(r"https://archive.ics.uci.edu/ml/machine-learning-databases/00292/Wholesale%20customers%20data.csv")
df = df.drop(columns=["Channel", "Region"])
df_og = df.copy()
df

In [ ]:
def plot_hist_cols():
    fig, axes = plt.subplots(3, 2, figsize=(10, 10))
    axes = axes.flatten()

    for i, col in enumerate(df.columns):
        sns.histplot(df[col], ax=axes[i])


plot_hist_cols()


In [ ]:
sns.pairplot(df)


In [ ]:
df = np.log(df + 1)
df_overview = df.copy()
plot_hist_cols()


In [ ]:
sns.pairplot(df)
# TODO: Add Correlation Plot and a column with total spendings
# Barplot/Pie chart with spendings from total spendings


# 2 Detecting Anomalies
## Hard-Min Score

In [ ]:
nbrs = NearestNeighbors(n_neighbors=2, algorithm="ball_tree").fit(df)
distances, indices = nbrs.kneighbors(df)
df_overview["outlier_score_min"] = np.square(distances[:, 1])
df_overview.sort_values(by="outlier_score_min", ascending=False).head(10)

In [ ]:
print(distances.shape)
# distances: Zeile = sample, Spalte 1 = kürzeste Distanz
# Spalte 0 = alles 0
print(indices.shape)
# indices: erste Spalte 0-440; zweite Spalte: index zur kürzesten Distanz

In [ ]:
print(distances[0, 1], indices[0])
# quick double check
np.linalg.norm(df.iloc[0, :] - df.iloc[59, :])

In [ ]:
print(np.var(df_overview['outlier_score_min']))
display(df_overview[["outlier_score_min"]].describe())
sns.boxplot(df_overview["outlier_score_min"])
plt.show()
sns.histplot(df_overview["outlier_score_min"])
# want it to be centered/dense!

## Soft-Min Score/KDE approach

In [ ]:
nbrs = NearestNeighbors(n_neighbors=len(df), algorithm="ball_tree").fit(df)
distances, indices = nbrs.kneighbors(df)
distances = np.square(distances)
gamma = 1

def softmin(z, gamma):
    # return -1 / gamma * np.log(1 / (len(z) - 1) * np.sum(np.exp(-gamma * z)))
    return -1 * np.log(1 / (len(z) - 1) * np.sum(np.exp(-gamma * z)))


df_overview["outlier_score_softmin"] = np.apply_along_axis(softmin, 1, distances[:, 1:], gamma)
df_overview.sort_values(by="outlier_score_softmin", ascending=False).head(10)

In [ ]:
print(distances.shape)

print(np.sum(distances == 0))
distances # aufsteigende Reihenfolge, indices enthält die zugehörigen Indices

In [ ]:
# TODO: Nebeneinander packe

print(np.var(df_overview['outlier_score_softmin']))
display(df_overview[["outlier_score_softmin"]].describe())
sns.boxplot(df_overview["outlier_score_softmin"])
plt.show()
sns.histplot(df_overview["outlier_score_softmin"])

In [ ]:
# visualize softmin function for a selected instance
instance = 338  # 338 has had a high outlier score
gammas = np.linspace(0.1, 2, 100)
g = sns.lineplot(
    x=gammas, y=[softmin(distances[instance, 1:], gamma) for gamma in gammas]
)
g.set_xlabel("gamma")
g.set_ylabel("softmin") 


## Bootstrap/Robustness estimate & choosing bandwidth

In [ ]:
# TODO should that be the same?
# import statistics
# statistics.fmean(np.exp(-gamma * distances[0, 1:]))
# softmin(distances[instance, 1:], gamma)

hohes gamma => weniger punkte einbezogen => höhere varianz, weniger bias

niedrigeres gamma => mehr punkte einbezogen (lim => 0 all uniform) => hoher bias, low variance

need 2 measures to compare: robustness vs. value of score (how much it discriminates between outliers and "regular" points); check picture from Montavon. Visual examination/optimal point. Compare to hard min


Cluster = Partition der Daten um sie zusammenzufassen, nicht unbedingt obvious

# SUGGESTED APPROACH ON TASK 2

In [ ]:
def softmin_og(z, gamma):
    return - (1/gamma)  * np.log(1 / (len(z) - 1) * np.sum(np.exp(-gamma * z)))

# NECESSARY TO USE ALL INSTANCES
# def clean_distances(z):
#     if z[0] == 0:
#         return z[1:]
#     else:
#         return z[0:-1]

In [ ]:
N_BOOTSTRAP = 100
gamma_range = np.linspace(0.1, 2, 20)

scores = np.full((len(df), N_BOOTSTRAP, len(gamma_range)), np.nan)


for i in range(0, N_BOOTSTRAP):

    sample = df.sample(frac=0.5)

    # USE ALL INSTANCES
    # nbrs = NearestNeighbors(n_neighbors=len(sample), algorithm="ball_tree").fit(sample)
    # distances, indices = nbrs.kneighbors(df)                                                    
    # distances = np.apply_along_axis(clean_distances, 1, distances)
    # distances = np.square(distances)
    
    # for j, gamma in enumerate(gamma_range):
    #     scores[:, i, j] = np.apply_along_axis(softmin_og, 1, distances, gamma)


    # TODO all instanes or only those in sample???
    # USE ONLY IN SAMPLE
    nbrs = NearestNeighbors(n_neighbors=len(sample), algorithm="ball_tree").fit(sample)
    distances, indices = nbrs.kneighbors(sample)
    distances = np.square(distances[:, 1:])
    
    for j, gamma in enumerate(gamma_range):
        scores[sample.index, i, j] = np.apply_along_axis(softmin_og, 1, distances, gamma)



In [ ]:
# check different when using all vs. only in sample
# spread = np.var(scores, axis=1)
# spread_only = np.nanvar(scores_only, axis=1)
# display(pd.DataFrame(np.linalg.norm(spread - spread_only, axis=1)).describe())

# verify normal distribution arround N_BOOTSTRAP * frac
sns.histplot(np.sum(np.isnan(scores) == False, axis=1)[:,0])

In [ ]:
# instance 0, first bootstrap sample, all values for gamma
display(scores[0, 0])

# first bootstrap sample, all instances per gamma
for i in range(len(scores)):
    plt.plot(gamma_range, scores[i, 0], linewidth=0.3)
plt.show()

In [ ]:
# calculate the spread over bootstrap dimension
spread = np.nanvar(scores, axis=1)

display(spread.shape)
print("Vars of instance 0:")
display(spread[0])

print("Vars of instance 338:")
display(spread[338])

# high values (variances) => anomaly score varied more in bootstrap samples => outlier scoring of instance is not so robust (it varies with sample)

# we want detection to be robust => so to have a low variance during bootstrap experiments

# all spreads per instances
for i in range(len(scores)):
    plt.plot(gamma_range, spread[i], linewidth=0.3)
plt.show()


In [ ]:
# gamma high => less points taken into account => on average higher variance (and less robust, right?)
g = sns.scatterplot(x=gamma_range, y=np.mean(np.nanvar(scores, axis=1), axis=0))
g.set_xlabel('gamma')
g.set_ylabel('Mean(Var(scores))')
plt.show()

In [ ]:
# average outlier score per sample, per gamma
avg_score = np.nanmean(scores, axis=1)

# TODO are the actual score values event meaningful? the range seems to depend on gamma 
# maybe the resulting ranking is more interesting?


display(avg_score.shape)
print("average outlier score of instance 0:")
display(avg_score[0])

print("average outlier score of instance 338:")
display(avg_score[338])

# high gamma => less values are taken into account => TODO seems to result in lower average outlier score (due to influence of gamma) but why?

# all avg_scores per instances
for i in range(len(scores)):
    plt.plot(gamma_range, avg_score[i], linewidth=0.3)
plt.show()


In [ ]:
g = sns.scatterplot(x=gamma_range, y=np.var(np.nanmean(scores, axis=1), axis=0))
g.set_xlabel('gamma')
g.set_ylabel('Var(Mean(scores))')
plt.show()

In [ ]:
g = sns.scatterplot(x=np.mean(np.nanvar(scores, axis=1), axis=0), y=np.var(np.nanmean(scores, axis=1), axis=0), hue=gamma_range)
g.set_xlabel('Mean(Var(scores))')
g.set_ylabel('Var(Mean(scores))')
plt.show()

In [ ]:
# TODO hard-min adden
# TODO ask about 1/gamma in softmin 

# var(mean(scores)) == discriminatory power ?
# mean(var(scores)) == model var?

## DIFFERENT APPROACH: SCORE DIFFERENCE IN 50/50 SPLIT
Does that solve the range problems caused by gamma?

In [ ]:
N_BOOTSTRAP = 100
gamma_range = np.linspace(0.1, 2, 40)

score_diff = np.zeros((len(df), N_BOOTSTRAP, len(gamma_range)))

hardmin_diff = np.zeros((len(df), N_BOOTSTRAP))

# NECESSARY TO USE ALL INSTANCES
def clean_distances(z):
    if z[0] == 0:
        return z[1:]
    else:
        return z[0:-1]


for i in range(0, N_BOOTSTRAP):

    # 50/50 split
    sample = df.sample(frac=0.5)
    rest = df.drop(sample.index)

    # USE ALL INSTANCES
    nbrs = NearestNeighbors(n_neighbors=len(sample), algorithm="ball_tree").fit(sample)
    distances1, _ = nbrs.kneighbors(df)
    distances1 = np.apply_along_axis(clean_distances, 1, distances1)
    distances1 = np.square(distances1)

    nbrs = NearestNeighbors(n_neighbors=len(rest), algorithm="ball_tree").fit(rest)
    distances2, _ = nbrs.kneighbors(df)
    distances2 = np.apply_along_axis(clean_distances, 1, distances2)
    distances2 = np.square(distances2)

    for j, gamma in enumerate(gamma_range):
        sm1 = np.apply_along_axis(softmin_og, 1, distances1, gamma)
        sm2 = np.apply_along_axis(softmin_og, 1, distances2, gamma)

        score_diff[:, i, j] = sm2 - sm1

    # hardmin
    hardmin1 = distances1[:, 1]
    hardmin2 = distances2[:, 1]

    hardmin_diff[:, i] = hardmin2 - hardmin1


In [ ]:
# hardmin has low bias but high variance (not so robust)
# TODO interpretation of that value corect?
np.mean(np.var(hardmin_diff, axis=0))


In [ ]:
# hardmin score # TODO interpretation of that value?
np.var(np.mean(hardmin_diff, axis=0))


In [ ]:
score_diff.shape

In [ ]:
spread = np.var(score_diff, axis=1)

for i in range(len(scores)):
    plt.plot(gamma_range, spread[i], linewidth=0.3)
plt.show()


In [ ]:
# robustness (how much to the scores vary over the bootstrap samples?)
# robustness increases with lower gamma (but high bias)
sns.scatterplot(x = gamma_range, y=np.mean(spread, axis=0))

In [ ]:
avg_score_diff = np.mean(score_diff, axis=1)

for i in range(len(scores)):
    plt.plot(gamma_range, avg_score_diff[i], linewidth=0.3)
plt.show()


In [ ]:
# measure discrimination 
# how good can we distinguish between the scores?
# higher variance is better here to identify the outliers
sns.scatterplot(x = gamma_range, y=np.var(avg_score_diff, axis=0))

In [ ]:
sns.scatterplot(x = np.mean(spread, axis=0), y=np.var(avg_score_diff, axis=0), hue=gamma_range)

# 3 Explaining Anomalies

In [ ]:
def relevance(x, yj):
    # Layer-wise Relevance Propagation
    Rji = np.zeros(x.shape)
    
    for j in range(len(x)):

        mask = np.full((len(x)), True)
        mask[j] = False

        z = np.square(np.linalg.norm(x - x[j], axis = 1))
        # y = softmin_og(z[mask], gamma)

        # 1.
        temp = np.exp(-gamma * z[mask])
        Rk = temp / np.sum(temp) * yj[j]

        # 2.
        nom = np.square(x-x[j])[mask]
        denom = np.square(np.linalg.norm(x - x[j], axis = 1))[mask]

        Rji[j,:] = np.sum(nom / denom[:, None] * Rk[:, None], axis=0)

    return Rji

In [ ]:
gamma = 1

# Calculate anomaly scores
nbrs = NearestNeighbors(n_neighbors=len(df), algorithm="ball_tree").fit(df)
distances, _ = nbrs.kneighbors(df)
yj = np.apply_along_axis(softmin_og, 1, np.square(distances[:, 1:]), gamma)

x = df.to_numpy()
# Layer-wise Relevance Propagation
Rji = relevance(x, yj)

# confirm conservation property
np.all(np.sum(Rji, axis=1) - yj <= 1e-14)

In [ ]:
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2, learning_rate='auto', init='pca', perplexity=30).fit_transform(df)

ax = sns.scatterplot(x=X_embedded[:,1], y=X_embedded[:,0], hue=yj, palette='flare')

norm = plt.Normalize(yj.min(), yj.max())
sm = plt.cm.ScalarMappable(cmap="flare", norm=norm)
sm.set_array([])

# Remove the legend and add a colorbar
ax.get_legend().remove()
ax.figure.colorbar(sm)

ax.set_ylabel("t-SNE 1")
ax.set_xlabel("t-SNE 2")

In [ ]:
Rji[338]

In [ ]:
instance = 338
display(df.iloc[instance])
g = sns.barplot(x=Rji[instance], y=list(map(str, list(zip(df_og.iloc[instance].index, df_og.iloc[instance].values)))))
g.set_xlim(0,np.max(Rji))

In [ ]:
# TODO WHY IS Fresh = 3 not relevant for outlier score?
df_og.describe()

## Bootstrap test

In [ ]:
gamma = 1

N_BOOTSTRAP = 100


Rji_BS = np.full((440, N_BOOTSTRAP, 6), np.nan)

for i in range(N_BOOTSTRAP):

    sample = df.sample(frac=0.5)

    nbrs = NearestNeighbors(n_neighbors=len(sample), algorithm="ball_tree").fit(sample)
    distances, _ = nbrs.kneighbors(sample)
    distances = np.square(distances[:, 1:])
    yj = np.apply_along_axis(softmin_og, 1, distances, gamma)
    
    Rji_BS[sample.index,i,:] = relevance(sample.to_numpy(), yj)

In [ ]:
display(np.nanvar(Rji_BS, axis=1))
pd.DataFrame(np.mean(np.nanvar(Rji_BS, axis=1), axis=1)).describe()
# TODO what to do with that?

In [ ]:
display(np.nanvar(Rji_BS[0], axis=0))
display(np.nanvar(Rji_BS[338], axis=0))


In [ ]:
np.nanvar(Rji_BS, axis=1)

In [ ]:
sns.boxplot(np.nanvar(Rji_BS, axis=1))